<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описаие данных

* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
import matplotlib.pyplot as plt

Загрузим данные и напишем функцию для основных характеристик данных.

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

def info(data_i):
    print('\033[1m' + 'Посмотрим на первые пять строк данных' + '\033[0m')
    display(data_i.head())
    print('______________________________________________________________')
    print('')
    print('\033[1m' + 'Посмотрим на общую информацию' + '\033[0m')
    display(data_i.info())
    print('______________________________________________________________')
    print('')
    print('\033[1m' + 'Посмотрим на основные статистические данные' + '\033[0m')
    display(data_i.describe())
    print('______________________________________________________________')
    print('')
    print('\033[1m' + 'Посмотрим, есть ли явные дубликаты' + '\033[0m')
    display(data_i.duplicated().sum())
    print('______________________________________________________________')
    print('')
    print('\033[1m' + 'Посмотрим на неявные дубликаты' + '\033[0m')
    display(data_i['id'].value_counts())

Выведем основные характеристики данных геологоразведки для 1 региона.

In [3]:
info(data_0)

Посмотрим на первые пять строк данных


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


______________________________________________________________

Посмотрим на общую информацию
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

______________________________________________________________

Посмотрим на основные статистические данные


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


______________________________________________________________

Посмотрим, есть ли явные дубликаты


0

______________________________________________________________

Посмотрим на неявные дубликаты


HZww2    2
74z30    2
TtcGQ    2
A5aEY    2
AGS9W    2
        ..
TltYZ    1
sbnp2    1
nmDNo    1
b0RjT    1
mXdxW    1
Name: id, Length: 99990, dtype: int64

Выведем основные характеристики данных геологоразведки для 2 региона.

In [4]:
info(data_1)

Посмотрим на первые пять строк данных


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


______________________________________________________________

Посмотрим на общую информацию
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

______________________________________________________________

Посмотрим на основные статистические данные


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


______________________________________________________________

Посмотрим, есть ли явные дубликаты


0

______________________________________________________________

Посмотрим на неявные дубликаты


LHZR0    2
5ltQ6    2
wt4Uk    2
bfPNe    2
7Ggnd    1
        ..
OVLkp    1
RyaWJ    1
Cpecj    1
e2Lvb    1
raui9    1
Name: id, Length: 99996, dtype: int64

Выведем основные характеристики данных геологоразведки для 3 региона.

In [5]:
info(data_2)

Посмотрим на первые пять строк данных


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


______________________________________________________________

Посмотрим на общую информацию
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

______________________________________________________________

Посмотрим на основные статистические данные


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


______________________________________________________________

Посмотрим, есть ли явные дубликаты


0

______________________________________________________________

Посмотрим на неявные дубликаты


VF7Jo    2
KUPhW    2
Vcm5J    2
xCHr8    2
92dGR    1
        ..
ARzZK    1
v97eO    1
wPPhv    1
N6F12    1
e04Le    1
Name: id, Length: 99996, dtype: int64

Итак, в таблице 4 признака и целевой признак. Тип данных у признаков - `object` `float64`.

Признаки:

* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек.

Целевой признак:

* `product` — объём запасов в скважине (тыс. баррелей).

#### Вывод

По основным характеристикам видно, что признаки распределены нормально. Данные практически готовы для начала решения задачи.

Признак `id` в нашей таблице не влияет на результат нашего исследования, поэтому можно его просто удалить.

Предварительно можно сказать, что данных достаточно для исследования. Перед началом работы необходимо провести предобработку данных. 

### Предобработка данных

Удалим признак `id` со всех датасетов, который не повлияет на результат исследования. 

In [6]:
data_0 = data_0.drop(['id'], axis=1)

In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [8]:
data_1 = data_1.drop(['id'], axis=1)

In [9]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [10]:
data_2 = data_2.drop(['id'], axis=1)

In [11]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


### Вывод

После предобработки количество признаков составило 3. Теперь можно приступить к решению задачи.

## Обучение и проверка модели

Поскольку работа происходит с 3мя таблицами, напишем функцию для основных этапов обучения моделей.

In [12]:
def allin(data_i):
    # поделим данные на обучающую и валидационную выборки в соотношении 75:25
    data_train, data_valid = train_test_split(data_i, test_size=0.25, random_state=12345) 
    
    # обучающие и валидационные переменные
    features_train = data_train.drop(['product'], axis=1) 
    target_train = data_train['product']
    features_valid = data_valid.drop(['product'], axis=1)
    target_valid = data_valid['product']
    print(f'Размеры тренировочных выборок: {features_train.shape, target_train.shape}.') 
    print(f'Размеры валидационных выборок: {features_valid.shape, target_valid.shape}.')
    print()
    
    # обучим модель и предскажем объемы запасов
    global pred_mean
    global RMSE
    
    model = LinearRegression() 
    model.fit(features_train, target_train) # обучим модель на обучающей выборке
    predictions = model.predict(features_valid) # предскажем ответы 
    pred_mean = predictions.mean() # посчитаем средний запас предсказанного сырья
    RMSE = mean_squared_error(target_valid, predictions)**0.5 # посчитаем RMSE модели
    print(f"Средний запас предсказанного сырья: {pred_mean:.5} тыс.баррелей.")
    print(f"RMSE модели валидационной выборки:  {RMSE:.5}.")
    return pred_mean, RMSE

In [13]:
pred_mean_0, RMSE_0 = allin(data_0)
print(pred_mean_0, RMSE_0)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 92.593 тыс.баррелей.
RMSE модели валидационной выборки:  37.579.
92.59256778438035 37.5794217150813


In [14]:
pred_mean_1, RMSE_1 = allin(data_1)
print(pred_mean_1, RMSE_1)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 68.729 тыс.баррелей.
RMSE модели валидационной выборки:  0.8931.
68.728546895446 0.893099286775617


In [15]:
pred_mean_2, RMSE_2 = allin(data_2)
print(pred_mean_2, RMSE_2)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 94.965 тыс.баррелей.
RMSE модели валидационной выборки:  40.03.
94.96504596800489 40.02970873393434


Применим функию ко всем регионам.

**Для 1 региона**

In [16]:
reg_0 = allin(data_0)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 92.593 тыс.баррелей.
RMSE модели валидационной выборки:  37.579.


**Для 2 региона**

In [17]:
reg_1 = allin(data_1)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 68.729 тыс.баррелей.
RMSE модели валидационной выборки:  0.8931.


**Для 3 региона**

In [18]:
reg_2 = allin(data_2)

Размеры тренировочных выборок: ((75000, 3), (75000,)).
Размеры валидационных выборок: ((25000, 3), (25000,)).

Средний запас предсказанного сырья: 94.965 тыс.баррелей.
RMSE модели валидационной выборки:  40.03.


### Вывод

В результате обучения модели и предсказания объемов запасов выявили "победителя" среднего объема запаса в каждом регионе. Им оказался 3 регион, затем идет 1 регион, а замыкает тройку 2 регион.

По показателю RMSE картина ровно противоположная. Выигрывает 2 регион, затем 3 регион, замыкает 3 регион. 


In [19]:
pd.DataFrame({'Средний запас предсказанного сырья (тыс.баррелей)': [f'{pred_mean_0:.5}', f'{pred_mean_1:.5}', f'{pred_mean_2:.5}']
              , 'RMSE': [f'{RMSE_0:.5}', f'{RMSE_1:.5}', f'{RMSE_2:.5}']}
             , index=['1 регион', '2 регион', '3 регион'])

,Средний запас предсказанного сырья (тыс.баррелей),RMSE
1 регион,92.593,37.579
2 регион,68.729,0.8931
3 регион,94.965,40.03


## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [20]:
BUDJET = 10000000000 # бюджет на разработку скважин в регионе
COUNT_OF_WALLS = 200 # 200 лучших точек для разработки
REV_PER_1000_BAR = 450000 # Доход с каждой единицы продукта (с тыс.бар.)
# считаем сумму для разработки одной точки
ratio_budget_count_of_walls = BUDJET/COUNT_OF_WALLS
# считаем объем сырья для безубыточной разработки новой скважины
count_bar = ratio_budget_count_of_walls/REV_PER_1000_BAR

print(f"Объем сырья для безубыточной разработки новой скважины:                 {count_bar:.5} тыс.баррелей.")

print(f'Разница безубыточного и среднепредсказанного объема сырья в 1 регионе:  {pred_mean_0-count_bar:.5} тыс.баррелей.')
print(f'Разница безубыточного и среднепредсказанного объема сырья во 2 регионе: {pred_mean_1-count_bar:.5} тыс.баррелей.')
print(f'Разница безубыточного и среднепредсказанного объема сырья в 3 регионе:  {pred_mean_2-count_bar:.5} тыс.баррелей.')

Объем сырья для безубыточной разработки новой скважины:                 111.11 тыс.баррелей.
Разница безубыточного и среднепредсказанного объема сырья в 1 регионе:  -18.519 тыс.баррелей.
Разница безубыточного и среднепредсказанного объема сырья во 2 регионе: -42.383 тыс.баррелей.
Разница безубыточного и среднепредсказанного объема сырья в 3 регионе:  -16.146 тыс.баррелей.


### Вывод

При расчете прибыли получили 111 тыс.бар. - это объем сырья для безубыточной разработки новой скважины. Самым убыточным получается 2 регион, затем 1 регион, и относительно менее убыточным по объему сырья получили в 3 регионе.

## Расчёт прибыли и рисков 

Напишем функцию расчета прибыли по выбранным скважинам и предсказаниям модели.

In [23]:
def profit(target, predictions, count):
    profit = 0 # переменная для суммирования прибыли 
    pr = predictions.sort_values(ascending=False) # сортируем предсказанные объемы сырья 
    selected = target[pr.index][:count] # выбираем ответы объемов сырья в соответствии 
                                        # с индексами предсказаний
    profit += (sum(selected)-count*count_bar)*450000 # считаем прибыль 
    return profit
# считаем прибыль в 1 регионе
data_0_valid['predictions'] = predictions_0
profit_0 = profit(data_0_valid['product'], data_0_valid['predictions'], 200)
print(f"Прибыль в 1 регионе от 200 топовых скважин:  {profit_0/10**6:.6} млн.")
# считаем прибыль во 2 регионе
data_1_valid['predictions'] = predictions_1
profit_1 = profit(data_1_valid['product'], data_1_valid['predictions'], 200)
print(f"Прибыль во 2 регионе от 200 топовых скважин: {profit_1/10**6:.6} млн.")
# считаем прибыль в 3 регионе
data_2_valid['predictions'] = predictions_2
profit_2 = profit(data_2_valid['product'], data_2_valid['predictions'], 200)
print(f"Прибыль в 3 регионе от 200 топовых скважин:  {profit_2/10**6:.6} млн.")

NameError: name 'predictions_0' is not defined

**Посчитаем риски и прибыль для каждого региона**

Посчитаем распределение прибыли техникой **Bootstrap** и оценки рисков. 

In [ ]:
def boots (target, predictions):
    state = np.random.RandomState(12345) # создаем объект RandomState()
    values = [] # сохраним значения прибылей в переменной values
    for i in range(1000): # создаем 1000 подвыборок
        target_subsample = target.sample(n=500, replace=True, random_state=state) # создаем подвыборокb
        pred_subsample = predictions[target_subsample.index] # выбираем предсказанные объемы сырья 
                                                             # в соответствии с индексами созданных подвыборок        
        values.append(profit(target_subsample, pred_subsample, 200)) # заполяем переменную values
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print(f'Вероятность убытков составляет                               {stats.percentileofscore(values, 0)} %.')
    print(f"Средняя прибыль составляет                                   {values.mean()/10**6:.5} млн.")
    print(f"Верхняя и нижняя граница доверительного интервала составляет {higher/10**6:.5} и {lower/10**6:.5} млн. соответственно.")
    print("")
    
print('\033[1m' + 'Для 1 региона' + '\033[0m')
boots(data_0_valid['product'], data_0_valid['predictions'])
print("")
print('\033[1m' + 'Для 2 региона' + '\033[0m')
boots(data_1_valid['product'], data_1_valid['predictions'])
print("")
print('\033[1m' + 'Для 3 региона' + '\033[0m')
boots(data_2_valid['product'], data_2_valid['predictions'])

## Вывод

Итоги выполненной работы:

* средний объем запаса больше всего в 3 регоине, затем в 1 и замыкает 2 сильноотстающий регион. Но по метрике RMSE лучшее качество у модели для 2 региона, а остальные два сильно хуже считает.

* 111 тыс.бар. - объем сырья для безубыточной разработки новой скважины. Наиболее убыточным оказался 2 регион, затем 1 регион, и относительно менее убыточный по объему сырья 3 регион.

* самым прибыльным регионом по 200 топовым скважинам оказался 1 регион, затем 3 и замыкающий 2 регион.

Исходя из последних расчетов получаем оптимальный вариант для разработки скважин во 2 регоине. Это показывает и вероятность убытков в 1% при условии максимального 2.5%, и средняя прибыль. По прибыльности 200 топовых скважин и объему запаса 2 регион уступает другим регионам. 

Подытожим. Несмотря на худшие позиции по объему запаса и прибыльности для топовых скважин, по метрике RMSE, вероятности убытков и средней прибыли перевешивает другие регионы по этим позициям, поэтому лучшим регионом для разработки скважин является 2 регион. 

